# Analyzing CIA factbook data with SQL

In this project, I will use data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), which contains country-specific demographic information.

## Setup

The code below connects this notebook to the database file:

In [38]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Examining the database

The code below returns a tabular summary of all the tables in our `factbook.db` database:

In [39]:
%%sql
SELECT *
    FROM sqlite_master
    WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Next, we examine the first 5 rows in the `facts` table:

In [40]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


This table contains country-specific demographic information. Next, I'll calculate some summary statistics.

## Summary statistics: country population and area

First, I return summary statistics on the population and population growth across countries

In [41]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
    FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


From the above, we see there is a country with population 0, and a country with population more than 7.2 billion. Now, I'll use subqueries to examine these countries in particular:

In [42]:
%%sql
SELECT name, population
    FROM facts
    WHERE population == (SELECT MIN(population) AS population FROM facts)

Done.


name,population
Antarctica,0


In [43]:
%%sql
SELECT name, population
    FROM facts
    WHERE population == (SELECT MAX(population) AS population FROM facts);

Done.


name,population
World,7256490011


In [44]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE population <> (SELECT MAX(population) AS population FROM facts);

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


Next, I calculate the average population and area:

In [45]:
%%sql
SELECT AVG(population) AS avg_pop,
       AVG(area) AS avg_area
    FROM facts;

Done.


avg_pop,avg_area
62094928.32231405,555093.546184739


## Identifying densely and sparsely populated countries

Next, I find countries that are **densely populated**, i.e. those that have above-average population values and below-average area values:

In [46]:
%%sql
SELECT name, population, area
    FROM facts
    WHERE population > (SELECT AVG(population) AS population FROM facts) 
        AND area < (SELECT AVG(area) AS area FROM facts);

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


Next, I find countries that are **sparsely populated**, i.e. those that have below-average population values and above-average area values. Since this returns a lot of countries, I then calculate the population-to-area ratio, and I return the countries with the top 7 largest ratios:

In [47]:
%%sql
SELECT name, population, area, population/area as pop_area_ratio
    FROM facts
    WHERE population < (SELECT AVG(population) AS population FROM facts) 
        AND area > (SELECT AVG(area) AS area FROM facts)
    ORDER BY pop_area_ratio DESC
    LIMIT 7;

Done.


name,population,area,pop_area_ratio
Burma,56320206,676578,83
Kenya,45925301,580367,79
Ukraine,44429471,603550,73
Tanzania,51045882,947300,53
Afghanistan,32564342,652230,49
South Africa,53675563,1219090,44
Colombia,46736728,1138910,41


## Identifying countries with a much higher death rate than birth rate

Next, I identify the countries where the death rate most outpaces the birth rate. To identify these countries, I compute the ratio of the birth to death rate for each country, and return the entries with the top 10 lowest ratios:

In [48]:
%%sql
SELECT name, birth_rate, death_rate, birth_rate/death_rate as birth_death_ratio
    FROM facts
    WHERE birth_death_ratio < 1
    ORDER BY birth_death_ratio
    LIMIT 10;

Done.


name,birth_rate,death_rate,birth_death_ratio
Bulgaria,8.92,14.44,0.6177285318559557
Serbia,9.08,13.66,0.664714494875549
Latvia,10.0,14.31,0.6988120195667366
Lithuania,10.1,14.27,0.7077785564120532
Hungary,9.16,12.73,0.7195600942655145
Monaco,6.65,9.24,0.7196969696969697
Slovenia,8.42,11.37,0.7405452946350044
Ukraine,10.72,14.46,0.7413554633471646
Germany,8.47,11.42,0.7416812609457093
Saint Pierre and Miquelon,7.42,9.72,0.7633744855967077


## Identifying countries with a much higher birth rate than death rate

Finally, I identify the countries where the birth rate most outpaces the death rate. To identify these countries, I again compute the ratio of the birth to death rate for each country, and return the entries with the top 10 highest ratios:

In [49]:
%%sql
SELECT name, birth_rate, death_rate, birth_rate/death_rate as birth_death_ratio
    FROM facts
    WHERE birth_death_ratio > 1
    ORDER BY birth_death_ratio DESC
    LIMIT 10;

Done.


name,birth_rate,death_rate,birth_death_ratio
Gaza Strip,31.11,3.04,10.233552631578947
Kuwait,19.91,2.18,9.133027522935778
Iraq,31.45,3.77,8.342175066312997
United Arab Emirates,15.43,1.97,7.83248730964467
Oman,24.44,3.36,7.273809523809525
Jordan,25.37,3.79,6.693931398416887
Solomon Islands,25.77,3.85,6.693506493506494
West Bank,22.99,3.5,6.5685714285714285
Qatar,9.84,1.53,6.431372549019607
Vanuatu,25.04,4.09,6.1222493887530565
